# XTTS Finetuning for Maltese

Author: [Mathieu Waharte](mailto:mathieu.waharte@universite-paris-saclay.fr)

## Requirements
- Python <3.11 for originial coqui's TTS and >=3.11 for TTS port
- HF_TOKEN
- Around 4GB for original files + 5GB per checkpoint

Installation takes around 6min (you may need to restart the session at the end)

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')

%cd /content/
%rm -rf Malta-TTS
!git clone https://github.com/Wubpooz/Malta-TTS.git

%cd Malta-TTS/FineTuning/NewLanguage
!pip install --upgrade pip
!pip install -r requirements.txt
!pip install tf-keras tensorflow-decision-forests tensorflow-text --upgrade

!python -c "import stanza; stanza.download('mt')"

/content
Cloning into 'Malta-TTS'...
remote: Enumerating objects: 1085, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 1085 (delta 76), reused 65 (delta 50), pack-reused 990 (from 1)
Receiving objects: 100% (1085/1085), 9.89 MiB | 15.05 MiB/s, done.
Resolving deltas: 100% (688/688), done.
/content/Malta-TTS/FineTuning/NewLanguage
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Ignoring TTS: markers 'python_version < "3.11"' don't match your environment
Ignoring trainer: markers 'python_version < "3.11"' don't match your environment
Ignoring coqpit: markers 'python_version < "3.11"' don't match your environment
Ignoring transformers: markers 'python_version < "3.11"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB

## Data Preparation

In [ ]:
# @title Save Metadata and Resample audio (takes 2:30min with 16 workers)

%cd /content/Malta-TTS/FineTuning/NewLanguage/

from prepare_maltese_dataset import load_and_resample

load_and_resample(output_dir='/content/drive/MyDrive/XTTS_Maltese_Data', dataset='Bluefir/MASRI_HEADSET_v2', sampling_rate=22050, num_workers=16, save_audio=False)

/content/Malta-TTS/FineTuning/NewLanguage
Loading dataset from Hugging Face...
Resampling to 22050Hz and saving...
Processing split: train (workers=16)


Resampling train: 100%|██████████| 3983/3983 [00:00<00:00, 4346.66it/s]


Saved 3983 entries to metadata_train.csv
Processing split: test (workers=16)


Resampling test: 100%|██████████| 996/996 [00:00<00:00, 4185.85it/s]


Saved 996 entries to metadata_eval.csv
Dataset saved!


In [ ]:
# @title Dataset repartition
from prepare_maltese_dataset import dataset_repartition

dataset_repartition('Bluefir/MASRI_HEADSET_v2')

Processing split: train
Processing split: test
Text length range: 1 - 188 characters
Audio duration range: 0.62 - 10.89 seconds
Average text length: 65.99 characters
Average audio duration: 4.82 seconds


## Finetuning

In [1]:
# @title Output Redirection

import os
import sys
_original_stdout = sys.stdout
_original_stderr = sys.stderr
_log_file = None

def output_redirect(output_path: str, redirect: bool = True):
  global _log_file
  if not redirect:
    sys.stdout = _original_stdout
    sys.stderr = _original_stderr
    if _log_file:
      _log_file.close()
      _log_file = None
  else:
    log_path = os.path.join(output_path, "full_training.log")
    # Clear the log file by opening in write mode and closing immediately
    if os.path.exists(log_path):
      with open(log_path, "w"):
        pass

    _log_file = open(log_path, "a", buffering=1)  # line-buffered

    class Tee(object):
      def __init__(self, *streams):
        self.streams = streams
      def write(self, data):
        for s in self.streams:
          s.write(data)
          s.flush()
      def flush(self):
        for s in self.streams:
          s.flush()

    sys.stdout = Tee(_original_stdout, _log_file)
    sys.stderr = Tee(_original_stderr, _log_file)

In [3]:
%env TOKENIZERS_PARALLELISM=false
%env OMP_NUM_THREADS=1
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

%env CUDA_VISIBLE_DEVICES=0
%env TORCH_ALLOW_TF32=1

import os

output_path = "/content/drive/MyDrive/XTTS_Maltese_Training"
metadata_train_path = "/content/drive/MyDrive/XTTS_Maltese_Data/metadata_train.csv"
metadata_eval_path = "/content/drive/MyDrive/XTTS_Maltese_Data/metadata_eval.csv"
language_code = "mt"
if not os.path.exists(output_path):
  os.makedirs(output_path, exist_ok=True)
output_redirect(output_path, False)

# 35min/epoch on one T4 with batch_size=1, grad_acumm=48, audio_length=255995, max_text=200, weight=1e-2
print(f"Finetuning for {language_code}...")
%cd /content/Malta-TTS/FineTuning/NewLanguage

from new_language_training_cli import main
from types import SimpleNamespace

# Memory optimization based on available VRAM?
# import torch
# if torch.cuda.is_available():
  # gpu_info = torch.cuda.get_device_properties(0)
  # print(f"GPU detected: {gpu_info.name}")
  # vram_gb = gpu_info.total_memory / 1024**3
  # print(f"   Memory: {vram_gb:.1f} GB")
  # print(f"   Compute Capability: {gpu_info.major}.{gpu_info.minor}")
  # if vram_gb < 16:
  #   print("⚠️ Low VRAM detected. Adjusting settings...")
  #   batch_size = 2
  #   gradient_accumulation = 126
  #   mixed_precision = True
  #   gradient_checkpointing = True
  # elif vram_gb < 24:
  #   batch_size = 3
  #   gradient_accumulation = 84
  #   mixed_precision = True
  # else:
  #   batch_size = 6
  #   gradient_accumulation = 42

args = SimpleNamespace(
    is_download=True,
    output_path=output_path,
    metadatas=[(metadata_train_path, metadata_eval_path, language_code)],
    num_epochs=3,
    batch_size=1,
    grad_acumm=48,
    min_frequency=2,
    max_new_tokens=800,
    max_audio_length=255995,
    max_text_length=200,
    weight_decay=1e-2,
    lr=5e-6,
    save_step=50000,
    version="main",
    language=language_code,
    forgetting_mitigation="none",  # or "LORA" or "Freeze"
    multi_gpu=False,
    optimizations=True,
    tf32=True
)

xtts_checkpoint, xtts_vocab, xtts_config, trainer_out_path, speaker_ref = main(args)
# lr = 5e-7, weight_decay = 0.1 for avoiding forgetting


print("Finetuning process completed!")
output_redirect(output_path, False)

env: TOKENIZERS_PARALLELISM=false
env: OMP_NUM_THREADS=1
env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
env: CUDA_VISIBLE_DEVICES=0
env: TORCH_ALLOW_TF32=1
Finetuning for mt...
[Errno 2] No such file or directory: '/content/Malta-TTS/FineTuning/NewLanguage'
/content


ModuleNotFoundError: No module named 'new_language_training_cli'

## Inference

In [ ]:
%cd /content/Malta-TTS/FineTuning/NewLanguage

import os
from inference import inference
import torchaudio

output_path = "/content/drive/MyDrive/XTTS_Maltese_Training/"
xtts_checkpoint = "/content/drive/MyDrive/XTTS_Maltese_Training/training/GPT_XTTS_FT-September-05-2025_08+01AM-3c88f84/checkpoint_3984.pth"
xtts_config = os.path.join(output_path, "config.json")
xtts_vocab = os.path.join(output_path, "vocab.json")

# Parameters
language_code = "en"
speaker_audio_file = "/content/drive/MyDrive/XTTS_Maltese_Data/wavs/MSRHS_M_11_P24U082_0147.wav" if language_code == "mt" else "/content/drive/MyDrive/english_speaker.mp3"

test_sentences = {
  'mt': [
    "Il-kelma Maltija 'bonġu' tfisser 'good morning'.",
    "Bonġu, kif int illum?",
    "Il-ħajja hija sabiħa.",
    "Grazzi ħafna tal-għajnuna tiegħek.",
    "Il-Milied qed joqrob malajr.",
    "Nispera li jkollok ġurnata tajba."
  ],
  'en': [
    "Hello, how are you today?",
    "This is a test of the new model.",
    "The weather is beautiful today.",
    "Thank you for your help.",
    "I hope you have a great day."
  ]
}
sentences_list = test_sentences.get(language_code)
if sentences_list is None:
    raise ValueError(f"No test sentences found for language code '{language_code}'")
sentences = sentences_list[0]


temperature = 0.7
length_penalty = 1.0
repetition_penalty = 10.0
top_k = 50
top_p = 0.8
LORA_trained = False

print("Starting inference...")
audio_waveform = inference(
  xtts_checkpoint=xtts_checkpoint,
  xtts_config=xtts_config,
  xtts_vocab=xtts_vocab,
  tts_text=sentences,
  speaker_audio_file=speaker_audio_file,
  lang_code=language_code,
  temperature=temperature,
  length_penalty=length_penalty,
  repetition_penalty=repetition_penalty,
  top_k=top_k,
  top_p=top_p,
  LORA_trained=LORA_trained
)
print("Inference completed!")

output_file = os.path.join(output_path, "output_"+language_code+".wav")
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
  os.makedirs(output_dir, exist_ok=True)
torchaudio.save(output_file, audio_waveform, sample_rate=24000)
print(f"Audio saved to {output_file}")

from IPython.display import Audio
Audio(output_file, rate=24000)

/content/Malta-TTS/FineTuning/NewLanguage
Starting inference...
[2025-09-05 10:37:15,523] [INFO] [real_accelerator.py:260:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-09-05 10:37:17,553] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
Loading model...
Config loaded.
Initializing model...
Loading checkpoint...
Detected standard model weights. Loading as base model...
[2025-09-05 10:39:22,027] [INFO] [logging.py:107:log_dist] [Rank -1] DeepSpeed info: version=0.17.5, git-hash=unknown, git-branch=unknown
[2025-09-05 10:39:22,060] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2025-09-05 10:39:22,061] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2025-09-05 10:39:22,429] [INFO] [lo

Using /root/.cache/torch_extensions/py312_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py312_cu124/transformer_inference/build.ninja...
/usr/local/lib/python3.12/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module transformer_inference...


Time to load transformer_inference op: 0.849553108215332 seconds
Model loaded successfully!
Computing speaker latents...
Speaker latents computed successfully!
Processing text...
Split into 1 sentences.
Final text chunks: 1
  Chunk 1: 24 chars - 'Hello, how are you today'
Running inference...


Processing sentences: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


Inference successful!
Inference completed!
Audio saved to /content/drive/MyDrive/XTTS_Maltese_Training/output_en.wav


/content/Malta-TTS/FineTuning/NewLanguage/inference.py:219: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(wav_chunks[0]).unsqueeze(0) #TODO remove torch.tensor?


## Further Improvements

In [ ]:
# @title Model Downloading
try:
  from google.colab import files
  files.download(output_file)
  print("📥 Download started!")
except ImportError:
  pass

### Widget use

In [ ]:
import ipywidgets as widgets
from IPython.display import display
# there is also Dropdown, Slider etc
text = widgets.Textarea(
   value="Il-kelma Maltija 'bonġu' tfisser 'good morning'.",
   placeholder='Type something',
   description='Text:',
   disabled=False
)
display(text)

Textarea(value="Il-kelma Maltija 'bonġu' tfisser 'good morning'.", description='Text:', placeholder='Type some…

In [ ]:
print(text.value)